In [5]:
def readGrid(path):
    with open(f"../assets/data/Pills/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n),  res

if __name__ == "__main__":
    m, n, grid = readGrid("problems/3_8x8")
    print(m, n)
    # print(grid)
    for g in grid:
        print(g)
    

8 8
['-', '4', '4', '4', '3', '7', '2', '4', '8']
['4', '0', '2', '2', '1', '3', '2', '2', '3']
['8', '3', '3', '4', '4', '1', '0', '2', '6']
['6', '4', '0', '0', '2', '4', '4', '2', '3']
['6', '1', '3', '1', '3', '0', '2', '2', '2']
['4', '1', '1', '1', '2', '3', '3', '2', '2']
['2', '1', '3', '2', '1', '2', '1', '2', '2']
['1', '1', '1', '4', '0', '3', '2', '2', '1']
['5', '1', '2', '2', '0', '4', '3', '3', '2']


In [19]:
from ortools.sat.python import cp_model as cp

def Pills_solver(X, Y, grid):
    model = cp.CpModel()
    solver = cp.CpSolver()
    x = {}
    clabels = grid[0][1:]
    pills_sum_dict = dict()
    
    fill_in_dict = dict() # 记录每个格子会被哪些药囊方格填充
    for i in range(X):
        for j in range(Y):
            fill_in_dict[i, j] = []
    # 计算每个胶囊的和

    rlabels = [grid[i][0] for i in range(1, len(grid))]
    
    # print(clabels, rlabels)
    
    for i in range(X):
        for j in range(Y - 2):
            x[0, i, j] = model.NewBoolVar(f"x_0_{i}_{j}")
            
            fill_in_dict[i, j].append(x[0, i, j])
            fill_in_dict[i, j + 1].append(x[0, i, j])
            fill_in_dict[i, j + 2].append(x[0, i, j])
            
            num_sum = int(grid[i + 1][j + 1]) + int(grid[i + 1][j + 2]) + int(grid[i + 1][j + 3])
            if num_sum not in pills_sum_dict:
                pills_sum_dict[num_sum] = [x[0, i, j]]
            else:
                pills_sum_dict[num_sum].append(x[0, i, j])

    for i in range(X - 2):
        for j in range(Y):
            x[1, i, j] = model.NewBoolVar(f"x_1_{i}_{j}")
            fill_in_dict[i, j].append(x[1, i, j])
            fill_in_dict[i + 1, j].append(x[1, i, j])
            fill_in_dict[i + 2, j].append(x[1, i, j])
            num_sum = int(grid[i + 1][j + 1]) + int(grid[i + 2][j + 1]) + int(grid[i + 3][j + 1])
            if num_sum not in pills_sum_dict:
                pills_sum_dict[num_sum] = [x[1, i, j]]
            else:
                pills_sum_dict[num_sum].append(x[1, i, j])

    for i in range(X):
        arr = []
        for j in range(Y):
            arr.append(int(grid[i + 1][j + 1]) * sum(fill_in_dict[i, j]))
        model.Add(sum(arr) == int(rlabels[i]))
        
    for j in range(Y):
        arr = []
        for i in range(X):
            arr.append(int(grid[i + 1][j + 1]) * sum(fill_in_dict[i, j]))
        model.Add(sum(arr) == int(clabels[j]))
    
    for i in range(1, X + 1):
        model.Add(sum(pills_sum_dict[i]) == 1)
    
    for i in range(X):
        for j in range(Y):
            model.Add(sum(fill_in_dict[i, j]) <= 1)
    
    status = solver.Solve(model)
    result = [["-" for _ in range(Y) ] for _ in range(X)]
    if status == cp.OPTIMAL:
        for k, v in x.items():
            if solver.Value(v) > 1e-3:
                if k[0] == 0:
                    result[k[1]][k[2]] = "x"
                    result[k[1]][k[2] + 1] = "x"
                    result[k[1]][k[2] + 2] = "x"
                else:
                    result[k[1]][k[2]] = "x"
                    result[k[1] + 1][k[2]] = "x"
                    result[k[1] + 2][k[2]] = "x"
        for i in range(X):
            for j in range(Y):
                if j != Y - 1:
                    print(result[i][j], end = " ")
                else:
                    print(result[i][j], end = "\n")

if __name__ == "__main__":
    m, n, grid = readGrid("problems/11_10x10")
    Pills_solver(m, n, grid)

- - - x x x x - - -
- - - - - - x - x -
- - - - - - x - x -
x x x - - - - x x -
- - - - - x - x - -
- - - - x x - x - -
- - - - x x - - - x
- - - - x - - - - x
- - - x x x - - - x
x x x - - - - - - -


In [9]:
a = [[1,2,3], [3,3,2]]


[1, 2, 3]
